# Description
In this notebook we export the parameter estimates as a DataFrame and a LaTeX table.

In [1]:
import arviz
import json
import pandas
import pathlib


DP_PROCESSED = pathlib.Path("processed")

### Read models and dataset

In [2]:
with open(DP_PROCESSED / "full_dataset_mle.json") as jfile:
    theta_dict = json.load(jfile)
idata = arviz.from_netcdf(DP_PROCESSED / "full_posterior.nc")
idata

Inference data with groups:
	> posterior
	> sample_stats
	> observed_data

Warmup iterations saved (warmup_*).

In [3]:
arviz.ess(idata)

<xarray.Dataset>
Dimensions:    (replicate: 28)
Coordinates:
  * replicate  (replicate) object 'A02' 'A03' 'A04' 'A05' ... 'D06' 'D07' 'D08'
Data variables:
    X0_mu      float64 9.272e+03
    F_offset   (replicate) float64 9.453e+03 9.686e+03 ... 9.558e+03 9.482e+03
    X0         (replicate) float64 9.434e+03 9.771e+03 ... 1.062e+04 1.091e+04
    S0         float64 9.607e+03
    Y_XS       float64 9.577e+03
    mu_max     float64 1.063e+04

### Create Dataframe and Save it

In [4]:
df = arviz.summary(idata, var_names="S0,mu_max,Y_XS,X0_mu,X0".split(","), hdi_prob=0.9)
rename_dict = {}
for k in df.index.values:
    if "[" in k:
        rvname, cindex = k.strip("]").split("[")
        cindex = int(cindex)
        cname = tuple(idata.posterior[rvname].coords.keys())[2+0]
        cval = idata.posterior.coords[cname].values[cindex]
        rename_dict[k] = f"{rvname}_{cval}"
df = df.rename(index=rename_dict)
df = df.assign(MLE=pandas.Series(theta_dict)).fillna("-")
cols = "MLE,mean,sd,hdi_5%,hdi_95%,r_hat"#,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail"
df = df[cols.split(",")]
df

,MLE,mean,sd,hdi_5%,hdi_95%,r_hat
S0,16.921204,15.230,0.103,15.061,15.400,1.0
mu_max,0.425082,0.425,0.001,0.424,0.426,1.0
Y_XS,0.672556,0.747,0.005,0.739,0.756,1.0
X0_mu,-,0.269,0.010,0.252,0.285,1.0
X0_A02,0.230797,0.266,0.053,0.179,0.348,1.0
X0_A03,0.335394,0.341,0.022,0.304,0.377,1.0
X0_A04,0.301141,0.299,0.006,0.289,0.310,1.0
X0_A05,0.266801,0.267,0.002,0.263,0.271,1.0
X0_A06,0.249569,0.250,0.002,0.247,0.252,1.0
X0_A07,0.252422,0.252,0.002,0.250,0.255,1.0


In [5]:
print(df.to_latex(bold_rows=True, longtable=False, label="tab_parameter_estimates"))

\begin{table}
\centering
\label{tab_parameter_estimates}
\begin{tabular}{llrrrrr}
\toprule
{} &        MLE &    mean &     sd &  hdi\_5\% &  hdi\_95\% &  r\_hat \\
\midrule
\textbf{S0    } &  16.921204 &  15.230 &  0.103 &  15.061 &   15.400 &    1.0 \\
\textbf{mu\_max} &   0.425082 &   0.425 &  0.001 &   0.424 &    0.426 &    1.0 \\
\textbf{Y\_XS  } &   0.672556 &   0.747 &  0.005 &   0.739 &    0.756 &    1.0 \\
\textbf{X0\_mu } &          - &   0.269 &  0.010 &   0.252 &    0.285 &    1.0 \\
\textbf{X0\_A02} &   0.230797 &   0.266 &  0.053 &   0.179 &    0.348 &    1.0 \\
\textbf{X0\_A03} &   0.335394 &   0.341 &  0.022 &   0.304 &    0.377 &    1.0 \\
\textbf{X0\_A04} &   0.301141 &   0.299 &  0.006 &   0.289 &    0.310 &    1.0 \\
\textbf{X0\_A05} &   0.266801 &   0.267 &  0.002 &   0.263 &    0.271 &    1.0 \\
\textbf{X0\_A06} &   0.249569 &   0.250 &  0.002 &   0.247 &    0.252 &    1.0 \\
\textbf{X0\_A07} &   0.252422 &   0.252 &  0.002 &   0.250 &    0.255 &    1.0 \\
\textbf{

In [6]:
%load_ext watermark
%watermark

Last updated: 2021-05-10T18:09:08.625702+02:00

Python implementation: CPython
Python version       : 3.7.9
IPython version      : 7.19.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 158 Stepping 10, GenuineIntel
CPU cores   : 12
Architecture: 64bit

